In [1]:
import os
import random
import copy
import re
import json
import jsonlines
import numpy as np
import pandas as pd
from tqdm import tqdm
import numpy as np
from collections import Counter

In [2]:
key_columns = ["input", "target", "answer_choices", "task_type", "task_dataset", "sample_id"]
templates = [
    "判断两个查询所表述的主题的匹配程度：\\n“[INPUT_TEXT_1]”，“[INPUT_TEXT_2]”。\\n选项：[LIST_LABELS]\\n答：",
    "我想知道下面两个搜索词的意思有多相同。\\n“[INPUT_TEXT_1]”，“[INPUT_TEXT_2]”。\\n选项：[LIST_LABELS]\\n答：",
    "下面两个句子的语义关系是？\\n“[INPUT_TEXT_1]”，“[INPUT_TEXT_2]”。\\n选项: [LIST_LABELS]\\n答：",
    "“[INPUT_TEXT_1]”和“[INPUT_TEXT_2]”表述的主题完全一致吗？\\n选项：[LIST_LABELS]\\n答：",
    "“[INPUT_TEXT_1]”和“[INPUT_TEXT_2]”的意思有多相似？\\n选项：[LIST_LABELS]\\n答：",
    "“[INPUT_TEXT_1]”，“[INPUT_TEXT_2]”。\\n这两句是一样的意思吗？\\n选项：[LIST_LABELS]\\n答：",
    "“[INPUT_TEXT_1]”，“[INPUT_TEXT_2]”。\\n这两句的语义关系是？\\n选项：[LIST_LABELS]\\n答："  
  ]

In [3]:
def read_data(data_path):

    data = []

    with jsonlines.open(data_path, "r") as f:
        for meta_data in f:
            data.append(meta_data)

    return data

In [4]:
def save_data(data_path, data):
    # write all_data list to a new jsonl
    with jsonlines.open(data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

In [9]:
def get_re(templates):
    '''get the re formula for input and labels'''
    re1, re2, re3 = [], [], []

    for temp in templates:

        re1.append(temp.replace("[INPUT_TEXT_1]", "([\s\S]*)").replace("[INPUT_TEXT_2]", "[\s\S]*").replace("[LIST_LABELS]", "[\s\S]*").replace("\\n答：", ""))
        re2.append(temp.replace("[INPUT_TEXT_2]", "([\s\S]*)").replace("[INPUT_TEXT_1]", "[\s\S]*").replace("[LIST_LABELS]", "[\s\S]*").replace("\\n答：", ""))
        re3.append(temp.replace("[LIST_LABELS]", "([\s\S]*)").replace("[INPUT_TEXT_1]", "[\s\S]*").replace("[INPUT_TEXT_2]", "[\s\S]*").replace("\\n答：", ""))

    return re1, re2, re3

In [10]:
train_data = read_data("train.json")
temp_re1, temp_re2, temp_re3 = get_re(templates)
input_list1, answer_list, input_list2 = [], [], []
anti_input = [] # check the sample that can't match the template

for meta_data in train_data:
    flag = True
    for i, temp in enumerate(temp_re1):
        if re.match(temp, meta_data["input"]):
            input_list1.append(re.match(temp, meta_data["input"])[1])
            answer_list.append(meta_data["target"])
            input_list2.append(re.match(temp_re2[i], meta_data["input"])[1])
            flag = False
            break
    if flag:    
        anti_input.append(meta_data["input"])

In [12]:
len(input_list1), len(set(input_list1)), len(answer_list), len(set(answer_list)), len(anti_input)

(3000, 772, 3000, 4, 0)

In [13]:
def get_input(raw_input1, raw_input2, raw_choices):

    meta_temp = random.choice(templates)
    new_input = copy.deepcopy(meta_temp)
    choice_str = ""
    for j, meta_choice in enumerate(raw_choices):
        if j == (len(raw_choices) - 1):
            choice_str = choice_str + meta_choice
        else:
            choice_str = choice_str + meta_choice + "，"
    new_input = new_input.replace("[INPUT_TEXT_1]", raw_input1)
    new_input = new_input.replace("[INPUT_TEXT_2]", raw_input2)
    new_input = new_input.replace("[LIST_LABELS]", choice_str)

    return new_input

In [14]:
raw_data = json.load(open("KUAKE-QQR_train.json", "r"))
aug_input_list1, aug_input_list2, aug_answer_list = [], [], []

for meta_d in tqdm(raw_data):
        
    aug_input_list1.append(meta_d["query1"])
    aug_input_list2.append(meta_d["query2"])
    aug_answer_list.append(meta_d["label"])

100%|██████████| 15000/15000 [00:00<00:00, 1478290.37it/s]


In [15]:
len(set(input_list1)), len(set(aug_input_list1)), len(set(input_list1+aug_input_list1))

(772, 797, 797)

In [36]:
new_input_list, new_answer_list, new_choice_list = [], [], []
for i in range(len(input_list)):
    if input_list[i] not in new_input_list:
        new_input_list.append(input_list[i])
        new_answer_list.append(answer_list[i])
        new_choice_list.append(choice_list[i])

In [37]:
len(new_input_list)

3622

In [38]:
random_index = random.sample(list(range(22304)), k=6000-3622)

In [39]:
for ri in random_index:
    new_input_list.append(aug_input_list[ri])
    new_answer_list.append(label_dict[aug_answer_list[ri]])
    n_choice = random.choice(list(range(1, 43)))
    meta_choice = random.sample(list(label_dict.values()), n_choice)
    if label_dict[aug_answer_list[ri]] not in meta_choice:
        meta_choice.append(label_dict[aug_answer_list[ri]])
    new_choice_list.append(meta_choice)

In [40]:
len(new_input_list)

6000

In [41]:
aug_data = []
count=0

for i in range(len(new_input_list)):

    meta_input = get_input(new_input_list[i], new_choice_list[i])
    meta_data = {"input": meta_input, "target": new_answer_list[i], "answer_choices": new_choice_list[i],
                    "task_type": "cls", "task_dataset": "CHIP-CTC", "sample_id": "train-"+str(count)}
    aug_data.append(meta_data)


In [42]:
save_data("train_aug.json", aug_data)